In [1]:
# Model Selection
# ===============
# This notebook compares the scores of different model algorithms, 
# preprocessing methods, and classification methods.
#
# The two best performing models are gradient boosting and radius neighbors.
# Radius neighbors performs better on standard informedness, diagnostic odds
# ratio, and outlier informedness. However, gradient boosting performs better 
# on cross-validation. This indicates that while radius neighbors performs
# better on the limited amount of data reserved for validation, however,
# gradient boosting appears to benefit more from increased data, which will
# be advantageous when we combine the training and validation datasets.
# Therefore, we choose gradient boosting with robust scaling as our model.
#
# Copyright 2020 Jerrad M. Genson
#
# This Source Code Form is subject to the terms of the Mozilla Public
# License, v. 2.0. If a copy of the MPL was not distributed with this
# file, You can obtain one at https://mozilla.org/MPL/2.0/.

library(tidyverse)

GIT_ROOT <- system2('git', args=c('rev-parse', '--show-toplevel'), stdout=TRUE)
DATA <- file.path(GIT_ROOT, 'data')
MODEL_DATA <- file.path(DATA, 'model_selection.csv')

scores <- read_csv(MODEL_DATA)
scores

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.4     ✔ dplyr   1.0.2
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.4.0     ✔ forcats 0.5.0

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


── Column specification ────────────────────────────────────────────────────────
cols(
  model = col_character(),
  preprocessing = col_character(),
  accuracy = col_double(),
  precision = col_double(),
  sensitivity = col_double(),
  specificity = col_double(),
  informedness = col_double(),
  dor = col_double(),
  ami = col_double(),
  `outlier informedness` = col_double(),
  `cv informedness` = col_double(),
  `mad informedness` = col_double(),
  `commit hash` = col_character()
)


Warning message:
“1 parsing failure.
row col   expected     actual                                                       f

model,preprocessing,accuracy,precision,sensitivity,specificity,informedness,dor,ami,outlier informedness,cv informedness,mad informedness,commit hash
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
rfc,robust scaling,0.8197,0.8421,0.6667,0.9189,0.5856,22.670,NA,0.5,0.5143,0.2317,f8469be
rfc,standard scaling,0.8033,0.8000,0.6667,0.8919,0.5586,16.500,0.2582,0.5,0.5143,0.2746,94c6579
rfc,pca,0.8033,0.7727,0.7083,0.8649,0.5732,15.540,0.2565,0.5,0.5071,0.2214,c34a598
rfc,isomap,0.7049,0.6071,0.7083,0.7027,0.4110,5.740,0.1113,0.3,0.5119,0.2000,cb0728e
rfc,lle,0.7377,0.6538,0.7083,0.7568,0.4651,7.556,0.1494,0.1,0.5786,0.2619,f708b98
rfc,feature agglomeration,0.7377,0.6538,0.7083,0.7568,0.4651,7.556,0.1494,0.3,0.5143,0.2000,3d44e37
rfc,nca,0.6721,0.5588,0.7917,0.5946,0.3863,5.573,0.1004,0.3,0.5071,0.1667,e9bdd96
rfc,factor analysis,0.7705,0.7273,0.6667,0.8378,0.5045,10.330,0.1929,0.5,0.4476,0.2095,87cb944
etc,robust scaling,0.7541,0.6957,0.6667,0.8108,0.4775,8.571,NA,0.5,0.5000,0.1619,9f5b916


In [2]:
best_informedness <- scores[scores$informedness == max(scores$informedness), ]
best_informedness

model,preprocessing,accuracy,precision,sensitivity,specificity,informedness,dor,ami,outlier informedness,cv informedness,mad informedness,commit hash
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
lrc,modified lle,0.8525,0.8947,0.7083,0.9459,0.6543,42.5,0.3875,0.5,0.4857,0.1762,a718db8


In [3]:
best_dor <- scores[scores$dor == max(scores$dor), ]
best_dor

model,preprocessing,accuracy,precision,sensitivity,specificity,informedness,dor,ami,outlier informedness,cv informedness,mad informedness,commit hash
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
lrc,modified lle,0.8525,0.8947,0.7083,0.9459,0.6543,42.5,0.3875,0.5,0.4857,0.1762,a718db8


In [4]:
best_sensitivity <- scores[scores$sensitivity == max(scores$sensitivity), ]
best_sensitivity

model,preprocessing,accuracy,precision,sensitivity,specificity,informedness,dor,ami,outlier informedness,cv informedness,mad informedness,commit hash
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
rfc,nca,0.6721,0.5588,0.7917,0.5946,0.3863,5.573,0.1004,0.3,0.5071,0.1667,e9bdd96


In [5]:
best_cv_informedness <- scores[scores$`cv informedness` == max(scores$`cv informedness`), ]
best_cv_informedness

model,preprocessing,accuracy,precision,sensitivity,specificity,informedness,dor,ami,outlier informedness,cv informedness,mad informedness,commit hash
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
qda,robust scaling,0.7869,0.7895,0.625,0.8919,0.5169,13.75,NA,0.5,0.6571,0.1571,eb89d93
qda,feature agglomeration,0.7869,0.7895,0.625,0.8919,0.5169,13.75,0.2257,0.5,0.6571,0.1571,44d8eda


In [6]:
second_best_informedness <- scores[scores$informedness == unique(sort(scores$informedness, TRUE))[2], ]
second_best_informedness

model,preprocessing,accuracy,precision,sensitivity,specificity,informedness,dor,ami,outlier informedness,cv informedness,mad informedness,commit hash
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
lrc,robust scaling,0.8361,0.85,0.7083,0.9189,0.6273,27.52,NA,0.5,0.5310,0.2167,87fb70c
lrc,isomap,0.8361,0.85,0.7083,0.9189,0.6273,27.52,0.3359,0.5,0.5143,0.2603,b49fe61
lrc,lle,0.8361,0.85,0.7083,0.9189,0.6273,27.52,0.3359,0.5,0.5310,0.2464,1866d34
lrc,nca,0.8361,0.85,0.7083,0.9189,0.6273,27.52,0.3359,0.5,0.5310,0.2917,3ad5091
qda,pca,0.8361,0.85,0.7083,0.9189,0.6273,27.52,0.3359,0.5,0.5982,0.2286,7519c68


In [7]:
third_best_informedness <- scores[scores$informedness == unique(sort(scores$informedness, TRUE))[3], ]
third_best_informedness

model,preprocessing,accuracy,precision,sensitivity,specificity,informedness,dor,ami,outlier informedness,cv informedness,mad informedness,commit hash
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
lrc,pca,0.8197,0.8095,0.7083,0.8919,0.6002,20.04,0.2931,0.5,0.5429,0.2083,1516719
